In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

recipes = pd.read_csv("recipes.csv")

In [3]:
ratings = pd.read_csv("interactions.csv")

In [5]:
print(recipes.columns)
print(ratings.columns)

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')


In [6]:
ratings

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [15]:
count_zero_ratings = ratings[ratings['rating'] == 0].count()
count_zero_ratings

user_id      60847
recipe_id    60847
date         60847
rating       60847
review       60847
dtype: int64

In [16]:
ratings.drop(ratings[ratings['rating'] == 0].index, inplace=True)

In [17]:
count_zero_ratings = ratings[ratings['rating'] == 0].count()
count_zero_ratings

user_id      0
recipe_id    0
date         0
rating       0
review       0
dtype: int64

In [30]:
ratings.isnull().sum()

user_id        0
recipe_id      0
date           0
rating         0
review       169
dtype: int64

In [31]:
ratings.dropna(inplace=True)

In [32]:
ratings

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132360,2002357020,82303,2018-12-05,5,Delicious quick thick chocolate sauce with ing...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [28]:
ratings_review = ratings[['review']]

In [24]:
ratings_df = ratings[['user_id', 'recipe_id', 'rating']]

In [20]:
recipes_df = recipes[['name', 'id', 'ingredients']]

In [21]:
recipes_df.rename(columns={'id': 'recipe_id'}, inplace=True)

In [22]:
# Save the new dataset to a new CSV file
recipes_df.to_csv('recipes_df.csv', index=False)

In [25]:
ratings_df.to_csv('ratings_df.csv', index=False)

In [29]:
ratings_review.to_csv('ratings_review.csv', index=False)

In [33]:
print(recipes_df.columns)
print(ratings_df.columns)

Index(['name', 'recipe_id', 'ingredients'], dtype='object')
Index(['user_id', 'recipe_id', 'rating'], dtype='object')


In [ ]:
# Check for duplicates
duplicates = recipes_df[recipes_df.duplicated()]
print(duplicates)

In [ ]:
recipes_df.isnull().sum()

In [ ]:
recipes_df.dropna(inplace=True)

In [ ]:
recipes_df.isnull().sum()